In [1]:
%run C:\Users\ender\PycharmProjects\pythonProject\DataScienceAssigments\lab2\lab2.ipynb

Setup Complete
-File 1 already downloaded
-File 2 already downloaded
-File 3 already downloaded
-File 4 already downloaded
-File 5 already downloaded
-File 6 already downloaded
-File 7 already downloaded
-File 8 already downloaded
-File 9 already downloaded
-File 10 already downloaded
-File 11 already downloaded
-File 12 already downloaded
-File 13 already downloaded
-File 14 already downloaded
-File 15 already downloaded
-File 16 already downloaded
-File 17 already downloaded
-File 18 already downloaded
-File 19 already downloaded
-File 20 already downloaded
-File 21 already downloaded
-File 22 already downloaded
-File 23 already downloaded
-File 24 already downloaded
-File 25 already downloaded
       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
0      1982     1  0.059  258.24  51.11  48.78  49.95      10
1      1982     2  0.063  261.53  55.89  38.20  47.04      10
2      1982     3  0.063  263.45  57.30  32.69  44.99      10
3      1982     4  0.061  265.10  53.96  28.62

In [2]:
from streamlit_jupyter import StreamlitPatcher
import streamlit as st
StreamlitPatcher().jupyter()

import pandas as pd
import urllib.request
from datetime import datetime
import os
import seaborn as sns
import matplotlib.pyplot as plt

st.title("Аналіз показників VCI/TCI/VHI")

province_dict = {
    1: "Вінницька", 2: "Волинська", 3: "Дніпропетровська", 4: "Донецька", 5: "Житомирська",
    6: "Закарпатська", 7: "Запорізька", 8: "Івано-Франківська", 9: "Київська", 10: "Кіровоградська",
    11: "Луганська", 12: "Львівська", 13: "Миколаївська", 14: "Одеська", 15: "Полтавська",
    16: "Рівненська", 17: "Сумська", 18: "Тернопільська", 19: "Харківська", 20: "Херсонська",
    21: "Хмельницька", 22: "Черкаська", 23: "Чернівецька", 24: "Чернігівська", 25: "АР Крим",
    26: "м. Київ", 27: "м. Севастополь"
}

if 'reset' not in st.session_state:
    st.session_state.reset = False
if 'choosen_value' not in st.session_state or st.session_state.reset:
    st.session_state.choosen_value = "VCI"
    st.session_state.region = list(province_dict.values())[0]
    st.session_state.weeks_range = (1, 15)
    st.session_state.years_range = (2000, 2005)
    st.session_state.sort_asc = False
    st.session_state.sort_desc = False
    st.session_state.reset = False

col1, col2 = st.columns([1, 2])

with col1:
    choosen_value = st.selectbox("Оберіть показник", ["VCI", "TCI", "VHI"], key="choosen_value")
    region = st.selectbox("Оберіть область", options=list(province_dict.values()), key="region")
    weeks_range = st.slider("Інтервал тижнів", 1, 52, key="weeks_range")
    years_range = st.slider("Інтервал років", 1982, 2023, key="years_range")
    sort_asc = st.checkbox("Сортувати за зростанням", key="sort_asc")
    sort_desc = st.checkbox("Сортувати за спаданням", key="sort_desc")

    if sort_asc and sort_desc:
        st.warning("Не можна обрати обидва сортування одночасно")

    if st.button("Скинути фільтри"):
        st.session_state.reset = True

with col2:
    region_index = list(province_dict.keys())[list(province_dict.values()).index(region)]
    filtered = converted_datafr[
        (converted_datafr['Region'] == region_index) &
        (converted_datafr['Week'].between(*weeks_range)) &
        (converted_datafr['Year'].between(*years_range))
    ]

    if sort_asc and not sort_desc:
        filtered = filtered.sort_values(by=choosen_value, ascending=True)
    elif sort_desc and not sort_asc:
        filtered = filtered.sort_values(by=choosen_value, ascending=False)

    tab1, tab2, tab3 = st.tabs(["Таблиця", "Графік", "Порівняння по регіонах"])

    with tab1:
        st.write("### Відфільтровані дані", filtered[["Year", "Week", choosen_value, "Region"]])

    with tab2:
        if not filtered.empty:
            plt.figure(figsize=(10, 5))
            sns.lineplot(data=filtered, x="Week", y=choosen_value, marker="o")
            plt.title(f"{choosen_value} у {region} ({years_range[0]} - {years_range[1]})")
            plt.xlabel("Тиждень")
            plt.ylabel(choosen_value)
            st.pyplot(plt)
        else:
            st.info("Немає даних для обраних фільтрів.")

    with tab3:
        compare_df = converted_datafr[
            (converted_datafr['Week'].between(*weeks_range)) &
            (converted_datafr['Year'].between(*years_range))
            ]

        if not compare_df.empty:
            plt.figure(figsize=(12, 6))

            # Групування та усереднення по тижнях для кожної області
            mean_df = compare_df.groupby(['Region', 'Week'])[choosen_value].mean().reset_index()

            # Побудова графіка з відтінками по областях
            sns.lineplot(data=mean_df, x="Week", y=choosen_value, hue="Region", linewidth=1)

            # Виділення вибраної області товстішою лінією
            selected_region_df = mean_df[mean_df['Region'] == region_index]
            sns.lineplot(data=selected_region_df, x="Week", y=choosen_value, color="black", linewidth=3,
                         label="Обрана область")

            plt.title(f"Порівняння {choosen_value} між регіонами ({years_range[0]}–{years_range[1]})")
            plt.xlabel("Тиждень")
            plt.ylabel(choosen_value)
            st.pyplot(plt)
        else:
            st.info("Немає даних для порівняння")


AttributeError: module 'streamlit' has no attribute 'experimental_data_editor'